# Problem 1 Notebook
By Timi Kola-Abiola and Aaron Baker

The goal is to create an effective bag-of-words representation of the reviews that is used to create a binary classifier. The binary classifer to be have the best performance on the provided test set. 

## Part 1: Bag-of-Words Design Decision Description
1. How did you “clean” the data? (handle punctuation, upper vs. lower case, numbers, etc)
Our plan is to remove make all words in the reviews lowercase and remove all punctuation and stop words. We use the CountVectorizer to remove the above from the reviews.  

In [111]:
import numpy as np
import pandas as pd
import os
import sklearn.feature_extraction.text as handler
import re
from retriever import *

In [112]:
x_train_df, y_train_df = return_x_and_y('data_reviews')

# Seperate the sites and reviews from the imported training data
x_sites = x_train_df['website_name']
x_reviews = x_train_df['text']
print(x_sites.shape)
print(x_reviews.shape)

Index(['website_name', 'text'], dtype='object')
(2400,)
(2400,)


In [113]:
# Got the Frequency of each unique word in the training set
vectorizer = handler.CountVectorizer(lowercase=True, stop_words='english')
X = vectorizer.fit_transform(x_reviews)
df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
df

,abandoned,abhor,ability,able,abound,abroad,absolute,absolutel,absolutely,absolutley,...,youtube,yucky,yukon,yum,yummy,yun,za,zero,zillion,zombie
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2395,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2396,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2397,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2398,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


2. How did you determine the final vocabulary set? did you exclude words, and if so how?
To determine the final vocabulary set for the Bag-of-Words, we will use TDIDF to find out how unique each word in the reviews are. Based on the TDIDF value, where a lower value means the word is not unique at all and vice versa, we will exclude the words that have the lowest TDIDF value. Our TDIDF threshold is 0.01, or all words in the training set that have a TDIDF value of 0.01 or small will be excluded from the bag-of-words

In [114]:
# feel free to change the min_df since we have not formally discussed the TDIDF threshold
transformer = handler.TfidfVectorizer(min_df=0.01, stop_words='english', lowercase=True)
tfidf_matrix = transformer.fit_transform(x_reviews)

df_tfidf = pd.DataFrame(tfidf_matrix.toarray(), columns=transformer.get_feature_names_out())
df_tfidf

,acting,amazing,bad,battery,best,better,came,case,characters,definitely,...,think,time,use,ve,waste,way,wonderful,work,works,worst
0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.696608,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,1.0,0.0,0.0,0.000000,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.570301,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2395,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2396,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2397,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2398,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


3. What was your final vocabulary size (or rough estimate of size(s), if size varies across folds because it depends on the training set)?
The final vocabulary size is 157 words for our Bag-of-Words representation of the training set.


In [100]:
len(df_tfidf.columns)

157

4. Did you use counts or binary values or something else?
We used counts to determine the frequency of unique words in the each sentence in the training set. 

5. How does your approach handle out-of-vocabulary words in the test set? (it’s fine if you just ignore them, but you should be aware of this)
Our approach ignores out-of-vocabulary words in the test set. 